# Bleach corrections evaluation
this notebook evaluate different bleach correction methods
- single exponential fit
- double exponential fit
- different regressions (robust)

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pynapple as nap

import matplotlib.pyplot as plt

import iblphotometry.preprocessing
import iblphotometry.plots

from utils import * # don't

from one.api import ONE

one = ONE(base_url="https://alyx.internationalbrainlab.org", mode='remote')

In [3]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 166

%load_ext autoreload
%autoreload 2

In [4]:
# select and load dataset
eid = "974a8a32-2156-4250-b9db-4298fc8daa65"
data_path = Path("/home/georg/data_local/raw_photometry_NE.pqt")
trials = one.load_dataset(eid, "alf/task_00/_ibl_trials.table.pqt")

# load trial data
raw_photometry = pd.read_parquet(data_path)
raw_photometry = nap.TsdFrame(raw_photometry.set_index("times"))

# restricting the fluorescence data to the time within the task +-1s
t_start = trials.iloc[0]["intervals_0"] - 1
t_stop = trials.iloc[-1]["intervals_1"] + 1

session_interval = nap.IntervalSet(t_start, t_stop)
raw_photometry = raw_photometry.restrict(session_interval)



/home/georg/anaconda3/envs/ibl/lib/python3.11/site-packages/one/api.py:300: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self._cache[table].loc[records.index[to_update], :] = records[to_update]
/home/georg/anaconda3/envs/ibl/lib/python3.11/site-packages/one/api.py:300: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self._cache[table].loc[records.index[to_update], :] = records[to_update]


In [5]:
F = raw_photometry['raw_calcium']
y, t = F.values, F.times()

from bleach_corrections import ExponDecayBleachingModel, DoubleExponDecayBleachingModel

bleaching_model = ExponDecayBleachingModel(method='L-BFGS-B')
F_corr = bleaching_model.bleach_correct(F)
# fig, axes = plt.subplots()
# axes.plot(F)

print(bleaching_model.calc_model_stats(F, use_kde=False, n_samples=-1))



{'r_sq': 0.9062994760670376, 'll': 1754190.8926128924, 'aic': -20.755036557449927}


In [12]:
%matplotlib qt5
bleaching_model = DoubleExponDecayBleachingModel()
F_corr = bleaching_model.bleach_correct(F)
fig, axes = plt.subplots()
axes.plot(F_corr, lw=0.5, alpha=0.5, color='g')
print(bleaching_model.popt)
# bleaching_model = DoubleExponDecayBleachingModel()
# F_corr = bleaching_model.bleach_correct_logspace(F)
# ax = plt.twinx(axes)
# ax.plot(F_corr, lw=0.5, alpha=0.5, color='r')



[1.86637126e-05 1.72255059e+03 1.84283267e-05 8.61275294e+02
 3.93149613e-03 3.45163820e-09]


libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [ ]:

# bleaching_model = DoubleExponDecayBleachingModel()
# _ = bleaching_model.bleach_correct(F)
# print(bleaching_model.calc_model_stats(F, use_kde=False, n_samples=-1))